In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import logging
import torch

from actioncard import Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch
from aiutils import load, save
from buyagenda import BigMoneyBuyAgenda
from config import GameConfig 
from constants import BUY
from enums import StartingSplit
from env import DefaultEnvironment
from player import HeuristicPlayer, load_players, PredictorMLPPlayer
from simulations import simulate

In [3]:
default_kingdom = [Chapel, Moat, Village, Militia, Moneylender, Smithy, CouncilRoom, Laboratory, Market, Witch]

In [4]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
# logger.setLevel(BUY)

## MLP Simulations ##

In [4]:
model_name = 'r-r-mcts-10k-score'
model_path = os.path.join(model_dir, model_name)
model = torch.load(model_path, map_location='cpu')
model

In [114]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
players = load_players(['MLP', 'BM'], [model_path], map_location='cpu', train=False)
agent = players[0]
env = DefaultEnvironment(config, players, logger=logger)

In [117]:
simulate(env, 1, None)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:root:Player 0, Turn 0: Estate
BUY:root:Player 1, Turn 1: Silver
BUY:root:Player 0, Turn 1: Estate
BUY:root:Player 1, Turn 2: Silver
BUY:root:Player 0, Turn 2: Estate
BUY:root:Player 1, Turn 3: Gold
BUY:root:Player 0, Turn 3: Estate
BUY:root:Player 1, Turn 4: Silver
BUY:root:Player 0, Turn 4: Estate
BUY:root:Player 1, Turn 5: Silver
BUY:root:Player 0, Turn 5: None
BUY:root:Player 1, Turn 6: Gold
BUY:root:Player 0, Turn 6: Estate
BUY:root:Player 1, Turn 7: Gold
BUY:root:Player 0, Turn 7: Copper
BUY:root:Player 1, Turn 8: Gold
BUY:root:Player 0, Turn 8: Estate
BUY:root:Player 1, Turn 9: Silver
BUY:root:Player 0, Turn 9: Estate
BUY:root:Player 1, Turn 10: Silver
BUY:root:Player 0, Turn 10: Copper
BUY:root:Player 1, Turn 11: Province
BUY:root:Player 0, Turn 11: Copper
BUY:root:Player 1, Turn 12: Gold
BUY:root:Player 0, Turn 12: Silver
BUY:root:Player 1, Turn 13: Province
BUY:root:Player 0, Turn 13: Copper
BUY:root:Player 1, Turn 14: Gold
BUY:root:Pla

Let's analyze the neural network outputs for some simple cases.

In [20]:
from state import ReducedStateFeature

In [21]:
skip = ReducedStateFeature.default_sandbox_feature()
copper = ReducedStateFeature.default_sandbox_feature()
silver = ReducedStateFeature.default_sandbox_feature()
estate = ReducedStateFeature.default_sandbox_feature()

In [22]:
copper[0] = 45
copper[7] = 8
copper

tensor([45., 10.,  8.,  8.,  8., 40., 30.,  8.,  0.,  3.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [23]:
silver[5] = 39
silver[7+5] = 1
silver

tensor([46., 10.,  8.,  8.,  8., 39., 30.,  7.,  0.,  3.,  0.,  0.,  1.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [24]:
estate[2] = 7
estate[7+2] = 4
estate

tensor([46., 10.,  7.,  8.,  8., 40., 30.,  7.,  0.,  4.,  0.,  0.,  0.,  0.,
         7.,  0.,  3.,  0.,  0.,  0.,  0.])

In [54]:
agent.model(copper), agent.model(silver), agent.model(estate)

TypeError: 'LogisticRegression' object is not callable

## Tree Simulations ##

In [6]:
from mcts import GameTree

In [22]:
tree_name = 'default-r-r-mcts'
tree_path = os.path.join(model_dir, tree_name)
rollout_name = 'default-mlog'
rollout_path = os.path.join(model_dir, rollout_name)

In [7]:
tree_name = 'r-dw-mcts-bm-100k-wl-base-default-ucb1'
rollout_name = 'mlog-10k-base-default-bm'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [8]:
tree = GameTree.load(tree_path, train=False)

In [121]:
tree._root.children[2].children[6].children[14]

Parent: Militia << (n: 1877, v: 1269.0, Q: 0.676 c: Witch) >> Children: [(0, 'Copper', '141', '0.177'), (1, 'Estate', '80', '0.025'), (2, 'Silver', '310', '0.329'), (3, 'Chapel', '503', '0.397'), (4, 'Moat', '538', '0.404'), (5, 'Village', '143', '0.199'), (6, 'None', '161', '0.208')]

In [9]:
# config = GameConfig(splits=[StartingSplit.Starting25Split, StartingSplit.Starting25Split], sandbox=True, num_players=2)
# splits = [StartingSplit.Starting25Split, StartingSplit.Starting25Split]
splits = [StartingSplit.StartingRandomSplit, StartingSplit.StartingRandomSplit]
config = GameConfig(splits=splits, sandbox=False, num_players=2, must_include=default_kingdom)
players = load_players(['UCT', 'DW'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=True)
# agent = load_players(['UCT'], [rollout_path], tree=tree, train=False, rollout_type='mlog', use_tree=False)[0]
# players = [agent, agent]
env = DefaultEnvironment(config, players, logger=logger)

In [12]:
# logger.setLevel(logging.ERROR)
logger.setLevel(BUY)

In [13]:
sim_data = simulate(env, 1, tree)

  0%|          | 0/1 [00:00<?, ?it/s]BUY:Selection
BUY:Player 0, Turn 0: Witch
BUY:Player 1, Turn 0: Silver
BUY:Selection
BUY:Player 0, Turn 1: Chapel
BUY:Player 1, Turn 1: Silver
BUY:Selection
BUY:Player 0, Turn 2: Copper
BUY:Player 1, Turn 2: Witch
BUY:Selection
BUY:Player 0, Turn 3: Estate
BUY:Player 1, Turn 3: Witch
BUY:Selection
BUY:Player 0, Turn 4: Copper
BUY:Player 1, Turn 4: None
BUY:Rollout
BUY:Player 0, Turn 5: Silver
BUY:Player 1, Turn 5: Gold
BUY:Rollout
BUY:Player 0, Turn 6: Gold
BUY:Player 1, Turn 6: Silver
BUY:Rollout
BUY:Player 0, Turn 7: Silver
BUY:Player 1, Turn 7: Silver
BUY:Rollout
BUY:Player 0, Turn 8: Gold
BUY:Player 1, Turn 8: Silver
BUY:Rollout
BUY:Player 0, Turn 9: Silver
BUY:Player 1, Turn 9: Gold
BUY:Rollout
BUY:Player 0, Turn 10: Silver
BUY:Player 1, Turn 10: Silver
BUY:Rollout
BUY:Player 0, Turn 11: Silver
BUY:Player 1, Turn 11: Silver
BUY:Rollout
BUY:Player 0, Turn 12: Gold
BUY:Player 1, Turn 12: Silver
BUY:Rollout
BUY:Player 0, Turn 13: Silver
BUY:Player

In [12]:
sim_data.game_df

,MaxScore,ProvinceWin,TimeElapsed,Tie
0,40.0,True,0.06635,False


In [97]:
data_dir = os.path.join(project_root, 'data')
data_name ='bm25-bm25-10k-scores'
data_path = os.path.join(data_dir, data_name)

In [98]:
save(data_path, sim_data.player_df)